In [67]:
import argparse
from webcam_utils import realtime_emotions
from prediction_utils import prediction_path

#!pip install easydict

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 


In [68]:
# for running realtime emotion detection
import easydict

def run_realtime_emotion():
    realtime_emotions()

# to run emotion detection on image saved on disk
def run_detection_path(path):
    return(prediction_path(path))

def emotion_detection():
    #parser = argparse.ArgumentParser()
    #parser.add_argument("func_name", type=str,
    #                    help="Select a function to run. <emo_realtime> or <emo_path>")
    #parser.add_argument("--path", default="saved_images/1.jpg", type=str,
    #                    help="Specify the complete path where the image is saved.")
    ## parse the args
    #args = parser.parse_args()

    img = Image.open("./image/input.jpg")
    #img = img.rotate(270)

    img.save('input.jpg')
    args = easydict.EasyDict({"func_name": "emo_path", "path":"input.jpg"}) #path를 바꿔주면 된다
    
    #img = Image.open("C:/Users/jinseong/data/input.jpg")
    
    draw = ImageDraw.Draw(img)
    _font = ImageFont.truetype("arial.ttf", int(img.size[0]/15))
    
    if args.func_name == "emo_realtime":
        result=run_realtime_emotion()
    elif args.func_name == "emo_path":
        result=run_detection_path(args.path)
    else:
        print("Usage: python main.py <function name>")
    
    return result
    

In [20]:
img = Image.open("input.jpg")
img.size

(261, 193)

In [16]:
img = Image.open("output.jpg")

FileNotFoundError: [Errno 2] No such file or directory: 'output.jpg'

In [69]:
import socketserver
import socket
import time


HOST = '211.57.9.212'
PORT = 9000

In [70]:
#https://lidron.tistory.com/42
class MyTcpHandler(socketserver.BaseRequestHandler):
    def handle(self):
        sock = self.request
        data_transferred = 0
        print('[%s] 연결됨' %self.client_address[0])

## 파일받아서 서버에 저장한다. (https://brownbears.tistory.com/207)

        IN_filename = 'input.jpg'# 클라이언트로 부터 파일이름을 이진 바이트 스트림 데이터를 일반 문자열로 변환을 전달받음
 
        data = sock.recv(1024)
        print ('first stack loaded')
        if not data:
            print('파일[%s]: 서버에 존재하지 않거나 전송중 오류발생' %IN_filename)
            return
 
        with open("./image/" + IN_filename , 'wb') as f:
            try:
                f.write(data)
                data_transferred += len(data)
                while  data:
                    data = sock.recv(1024)
                    f.write(data)
                    data_transferred += len(data)
                    #print (data_transferred, )
                    time.sleep(0.01)
            except Exception as e:
                print(e)
 
        print('파일[%s] 전송종료. 전송량 [%d]' %(IN_filename, data_transferred))
        
        result = emotion_detection()
        sock.sendall(result.encode('utf-8'))
        
        sock.close()
        print('소켓 닫습니다')
        return

In [74]:
def runServer():
    print(HOST+':'+str(PORT)+' 소켓 서버 시작')
    print('++++++파일 서버를 시작++++++')
    print("+++파일 서버를 끝내려면 'Ctrl + C'를 누르세요.")
 
    try:
        server = socketserver.TCPServer((HOST,PORT),MyTcpHandler)
        server.serve_forever()
    except KeyboardInterrupt:
        print('++++++파일 서버를 종료합니다.++++++')
 
 
runServer()
print('++++++파일 서버를 종료합니다.++++++')

211.57.9.212:9000 소켓 서버 시작
++++++파일 서버를 시작++++++
+++파일 서버를 끝내려면 'Ctrl + C'를 누르세요.
[211.57.9.212] 연결됨
first stack loaded
